Załaduj FIQA razem z relacjami

In [2]:
from datasets import load_dataset

fiqa_corpus = load_dataset("clarin-knext/fiqa-pl", name="corpus")
fiqa_queries = load_dataset("clarin-knext/fiqa-pl", name="queries")
fiqa_qrels = load_dataset("clarin-knext/fiqa-pl-qrels")

In [3]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from collections import Counter
import re

nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)

clean_token_pattern = re.compile(r"\W|\d|_")

texts = fiqa_corpus['corpus']['text'] + fiqa_queries['queries']['text']

token_counter = Counter()
for doc in tokenizer.pipe(texts, batch_size=500):
    for token in doc:
        cleaned_token = clean_token_pattern.sub("", token.text)
        if cleaned_token:
            token_counter[cleaned_token.lower()] += 1

token_counter

Counter({'w': 177313,
         'nie': 132154,
         'i': 126963,
         'na': 120492,
         'to': 116453,
         'z': 98189,
         'jest': 94338,
         'że': 90150,
         'się': 86403,
         'do': 67205,
         'jeśli': 52121,
         'a': 41506,
         'ale': 41325,
         'o': 38510,
         'są': 35946,
         'jak': 34795,
         'lub': 32862,
         'za': 32725,
         'aby': 31263,
         'od': 30211,
         'co': 30207,
         'może': 26590,
         'dla': 25950,
         'po': 25902,
         'tak': 25684,
         'które': 24100,
         'możesz': 23366,
         'czy': 22244,
         'tego': 21982,
         'tym': 21659,
         'ma': 20417,
         'być': 19355,
         'ponieważ': 18952,
         'przez': 17973,
         'tylko': 17347,
         'usd': 17257,
         'więc': 17081,
         'niż': 16946,
         'ich': 14669,
         'pieniądze': 14131,
         'więcej': 13928,
         'gdy': 13681,
         'jako': 131

In [4]:
fiqa_queries

DatasetDict({
    queries: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 6648
    })
})

In [66]:
import random
import string

def distort_token(token):
    word = token.text
    idx = random.randint(0, len(word) - 1)
    
    distortion_type = random.randint(0, 2)
    new_word = word
    if distortion_type == 0 and len(word) > 1:
        new_word =  word[:idx] + word[idx + 1:]
    elif distortion_type == 1:
        random_letter = random.choice(string.ascii_lowercase)
        new_word = word[:idx] + random_letter + word[idx:]
    elif distortion_type == 2:
        random_letter = random.choice(string.ascii_lowercase)
        new_word =  word[:idx] + random_letter + word[idx + 1:]
    return new_word

def distort_query_tokens(query):
    doc = tokenizer(query)
    tokens = list(doc)
    
    token_idx = random.randint(0, len(tokens) - 1)
    distorted_token = distort_token(tokens[token_idx])
    
    tokens[token_idx] = distorted_token
    return " ".join(str(token) for token in tokens)


distorted_queries_text = [distort_query_tokens(query) for query in fiqa_queries['queries']['text']]

In [106]:
distorted_queries_text

['Co jest uważane zqa wydatek służbowy w podróży służbowej?',
 'Wydatki służbowe - ubezpieczenie samocodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
 'Rozpoczęcie nowemgo biznesu online',
 '„Dzień roboczy i „termin płatności” rachunków',
 'Nowy właściciel frmy – Jak działają podatki dla firmy i osoby fizycznej?',
 'Hobby kontra fiznes',
 'Czeki osobiste zamiast firowych',
 'Czy amerykański kodeks podaxtkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?',
 'Jak mogę zarejestrować fimę w Wielkiej Brytanii bez podawania adresu firmy?',
 'Czym saą „podstawy biznesowe”?',
 'Strata na inwestycjach biznesdwych z poprzedniego roku',
 'Japk mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy, która ma 0 USD dochodu?',
 'Czy zakup samochodu dla firmy za pomocą kredytu biznesowego zostałby uznany za wydatek biznesow?',
 'Odliczanie strat biznesowych z ostatnich lat (nieudokumentoanych)',
 '30% udztiału w biz

In [71]:
distorted_queries = fiqa_queries['queries'].map(lambda x, idx: {"text": distorted_queries_text[idx]}, with_indices=True)


Map: 100%|██████████| 6648/6648 [00:00<00:00, 39955.54 examples/s]


In [73]:
distorted_queries['text']

['Co jest uważane zqa wydatek służbowy w podróży służbowej?',
 'Wydatki służbowe - ubezpieczenie samocodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
 'Rozpoczęcie nowemgo biznesu online',
 '„Dzień roboczy i „termin płatności” rachunków',
 'Nowy właściciel frmy – Jak działają podatki dla firmy i osoby fizycznej?',
 'Hobby kontra fiznes',
 'Czeki osobiste zamiast firowych',
 'Czy amerykański kodeks podaxtkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?',
 'Jak mogę zarejestrować fimę w Wielkiej Brytanii bez podawania adresu firmy?',
 'Czym saą „podstawy biznesowe”?',
 'Strata na inwestycjach biznesdwych z poprzedniego roku',
 'Japk mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy, która ma 0 USD dochodu?',
 'Czy zakup samochodu dla firmy za pomocą kredytu biznesowego zostałby uznany za wydatek biznesow?',
 'Odliczanie strat biznesowych z ostatnich lat (nieudokumentoanych)',
 '30% udztiału w biz

In [81]:
def prepare_fiqa_qrels():
    query_to_corpus_dict = {}

    subsets = ['test', 'validation', 'train']

    for subset in subsets:
        for item in fiqa_qrels[subset]:
            if item['query-id'] not in query_to_corpus_dict:
                query_to_corpus_dict[item['query-id']] = {}

            query_to_corpus_dict[item['query-id']][item['corpus-id']] = item['score']

    for query_id in query_to_corpus_dict:
        sorted_corpuses_by_score = dict(sorted(query_to_corpus_dict[query_id].items(), key=lambda item: item[1]))
        query_to_corpus_dict[query_id] = sorted_corpuses_by_score

    return query_to_corpus_dict

def prepare_existing_fiqa_queries(query_to_corpus_dict):
    queries_dict = {}

    queries_dataset = fiqa_queries['queries']
    for entry in queries_dataset:
        if int(entry['_id']) in query_to_corpus_dict.keys():
            queries_dict[int(entry['_id'])] = entry['text']

    return queries_dict


query_to_corpus_dict = prepare_fiqa_qrels()
queries_dict = prepare_existing_fiqa_queries(query_to_corpus_dict)

In [ ]:
dcg_limit = 10

def find_for_phrase(search_phrase, search_field, custom_analyzer):
    search_query = {
        "size": dcg_limit,
        "query": {
            "match": {
                search_field: {
                    "query": search_phrase,
                }
            }
        }
    }

    if custom_analyzer:
        search_query['query']['match'][search_field]['analyzer'] = custom_analyzer

    response = requests.get(f"{index_url}/_search", headers={"Content-Type": "application/json"}, data=json.dumps(search_query))

    if response.status_code == 200:
        search_results = response.json()
        return dict(list(map(lambda hit: (int(hit['_id']), float(hit['_score'])), search_results["hits"]["hits"])))
    else:
        print(f"Search failed: {response.text}")




In [ ]:
import math

relevant_doc_number = 0
relevant_docs = []

def calculate_dcg(docs, docs_scoring):
    sum = 0
    relevant_doc_id = None
    for i, doc_id in enumerate(docs):
        if doc_id in docs_scoring.keys():
            sum += (2**docs_scoring[doc_id]-1)/(math.log2(i+1+1))
            if i == relevant_doc_number:
                relevant_doc_id = doc_id
    return sum, relevant_doc_id

def calculate_ndcgs(search_field, custom_analyzer):
    
    ndcgs = []

    for q_id, q_text in queries_dict.items():
        ideal_search = list(query_to_corpus_dict[q_id].keys())[:dcg_limit]
        idcg, _ = calculate_dcg(ideal_search, query_to_corpus_dict[q_id])

        real_search_with_scores = find_for_phrase(q_text, search_field, custom_analyzer)
        dcg, relevant_doc_id = calculate_dcg(real_search_with_scores.keys(), query_to_corpus_dict[q_id])

        ndcgs.append(dcg/idcg)

        if relevant_doc_id:
            relevant_docs.append((relevant_doc_id, real_search_with_scores[relevant_doc_id], real_search_with_scores.values(), q_text))

    return ndcgs

import matplotlib.pyplot as plt
import numpy as np
def present_results(ndcgs):

    zeros = []
    non_zeros = []

    for ndcg in ndcgs:
        if ndcg == 0:
            zeros.append(ndcg)
        else:
            non_zeros.append(ndcg)

    labels = ['>0 Results', '0 Results']
    sizes = [len(non_zeros), len(zeros)]

    plt.pie(sizes, labels=labels, autopct='%1.1f%%')
    plt.title('NDCG@5 Results Proportion: 0 to >0')
    plt.show()

    plt.hist(non_zeros, bins=10, edgecolor='black')
    plt.title("Histogram of Non-Zero NDCG@5")
    plt.xlabel("Score")
    plt.ylabel("Number of Documents")
    plt.show()

    print(f"NDCG@5 Mean: {np.mean(ndcgs)} and Std: {np.std(ndcgs)}")
    print(f"NDCG@5 > 0 Mean: {np.mean(non_zeros)} and Std: {np.std(non_zeros)}")



In [88]:
def read_morfeusz_into_set(file_path):
    correct_words = set()

    with open(file_path, "r", encoding="utf-8") as file:
        for line_number, line in enumerate(file):
            if line_number < 31:
                continue
            word = line.split("\t")[0]
            correct_words.add(word.lower())
    return correct_words

correct_words = read_morfeusz_into_set("morfeusz/polimorf-20241117.tab")
len(correct_words)


5348627

In [99]:
import re

def split_to_words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(token_counter.values())): 
    "Probability of `word`."
    return token_counter[word] / N

def correct(word): 
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in token_counter)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
skip_word_pattern = re.compile(r"(\d|_)+")

def correct_queries_text(query):
    words = split_to_words(query)
    corrected_query = query
    for word in words:
        if word not in correct_words:
            if skip_word_pattern.match(word):
                continue
            corrected_word = correct(word)
            corrected_query = corrected_query.replace(word, corrected_word)
    return corrected_query    
            

corrected_queries_text = [correct_queries_text(query) for query in distorted_queries['text']]

In [113]:
corrected_queries = distorted_queries.map(lambda x, idx: {"text": corrected_queries_text[idx]}, with_indices=True)

Map: 100%|██████████| 6648/6648 [00:00<00:00, 56652.95 examples/s]


In [114]:
corrected_queries['text']

['Co jest uważane za wydatek służbowy w podróży służbowej?',
 'Wydatki służbowe - ubezpieczenie samochodu podlegające odliczeniu za wypadek, który wydarzył się podczas podróży służbowej',
 'Rozpoczęcie nowego biznesu online',
 '„Dzień roboczy i „termin płatności” rachunków',
 'Nowy właściciel firmy – Jak działają podatki dla firmy i osoby fizycznej?',
 'Hobby kontra biznes',
 'Czeki osobiste zamiast firmowych',
 'Czy amerykański kodeks podatkowy wymaga, aby właściciele małych firm liczyli zakupy biznesowe jako dochód osobisty?',
 'Jak mogę zarejestrować firmę w Wielkiej Brytanii bez podawania adresu firmy?',
 'Czym są „podstawy biznesowe”?',
 'Strata na inwestycjach biznesowych z poprzedniego roku',
 'Japk mogę oszacować podatki biznesowe / opłaty za zgłoszenie dla firmy, która ma 0 USD dochodu?',
 'Czy zakup samochodu dla firmy za pomocą kredytu biznesoweego zostałby uznany za wydatek biznesowe?',
 'Odliczanie strat biznesowych z ostatnich lat (nieudokumentowanych)',
 '30% udziału w b